In [1]:
import numpy as np
import codecs
from dataclasses import dataclass
from typing import List, Dict, Any

import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor, nn
import random

# Ensuring reproducibility
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

random.seed(seed)

import sys
sys.path.insert(1, '/work/nlp-project')
from scripts.read_write_data import load_data
from models.classes import DataIterator, Batch, BaselineBiLSTM, F1_evaluator

import gensim.models
GoogleEmbs = gensim.models.KeyedVectors.load_word2vec_format(
                                '/work/nlp-project/models/GoogleNews-50k.bin', binary=True)

In [10]:
# TRAIN_PATH = "nlp-project/data/processed/train.conll"
TRAIN_PATH = "/work/nlp-project/data/processed/train_splits/labeled.conll"
DEV_PATH = "nlp-project/data/processed/dev.conll"

x_train, y_train, _, _ = load_data(TRAIN_PATH)
x_dev, y_dev, _, _ = load_data(DEV_PATH)

In [3]:
model = BaselineBiLSTM(hidden_size=8)
model.fit(x_train, y_train, dev=(x_dev, y_dev), 
          print_metrics=False, 
          learning_rate=0.01, 
          epochs=5)

Epoch 0, train: 0.005, dev: 0.000
Epoch 1, train: 0.000, dev: 0.000
Epoch 2, train: 0.000, dev: 0.000
Epoch 3, train: 0.153, dev: 0.726
Epoch 4, train: 0.821, dev: 0.780


In [4]:
"""
hidden_size=5, lr=0.05 --- Epoch 22, train: 0.861, dev: 0.774
hidden_size=10, lr=0.05 --- Epoch 22, train: 0.849, dev: 0.780
hidden_size=10, lr=0.05 --- Epoch 9, train: 0.849, dev: 0.778
hidden_size=9, lr=0.01 --- Epoch 25, train: 0.919, dev: 0.800

source for saving and loading: https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training
"""

'\nhidden_size=5, lr=0.05 --- Epoch 22, train: 0.861, dev: 0.774\nhidden_size=10, lr=0.05 --- Epoch 22, train: 0.849, dev: 0.780\nhidden_size=10, lr=0.05 --- Epoch 9, train: 0.849, dev: 0.778\nhidden_size=9, lr=0.01 --- Epoch 25, train: 0.919, dev: 0.800\n\nsource for saving and loading: https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training\n'

Saving and loading model:

In [5]:
SAVE_PATH = "/work/nlp-project/models/Baseline.pt"
torch.save(model, SAVE_PATH)

In [6]:
model2 = torch.load(SAVE_PATH)
model2.dev_f1_log

[0, 0, 0, 0.7259747391543109, 0.7802139037433156]

In [7]:
model2.evaluate(x_dev, y_dev)

F1 score: 0.780
Metrics: 1611 ACT, 2129 POS, 1459 COR, 0 PAR, 822 INC (670 MIS, 152 SPU)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b2f14aee-af04-4db5-af55-57a3a58b9f40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>